## Initialization of the notebook with installation and import of correct versions of libraries.

In [1]:
!pip install torchinfo
!pip install brevitas
!pip install qonnx
!pip install onnxoptimizer

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu May 15 13:13:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
import numpy as np
import os
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import random
import seaborn as sns
import pandas as pd
import gc
import cv2

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

In [6]:
import brevitas.nn as qnn
from brevitas.quant import Int8ActPerTensorFloat


In [7]:
read_small = True

In [8]:
if read_small:
  !unzip drive/MyDrive/TecnosensSondaOttica/DatasetTS_MA_small.zip -d ./dataset
else:
  !unzip drive/MyDrive/TecnosensSondaOttica/DatasetTS_MA_FT_medium.zip -d ./dataset

unzip:  cannot find or open drive/MyDrive/TecnosensSondaOttica/DatasetTS_MA_small.zip, drive/MyDrive/TecnosensSondaOttica/DatasetTS_MA_small.zip.zip or drive/MyDrive/TecnosensSondaOttica/DatasetTS_MA_small.zip.ZIP.


In [9]:
datasetPath = "drive/MyDrive/HPPS_Nico/HPPS/Project/ModelRegression/Dataset"

In [10]:
%ls

drive/  sample_data/


## Setting a seed on all frameworks for reproducibility

In [11]:
# Random seed for reproducibility
seed = 1234

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Load data and preprocessing

## Generate dataset

In [ ]:
labels_df = pd.read_csv(datasetPath+"/labels_df2.csv")
#if not read_small:
  #labels_df = pd.read_csv("../drive/MyDrive/TecnosensSondaOttica/labels_df_ft.csv", index_col=False)

In [ ]:
X_train = []
Y_train = []
X_val = []
Y_val = []
X_test = []
Y_test = []

#dir = "DatasetTS_MA_FT_medium"
if read_small:
  dir = datasetPath+"/DatasetTS_MA_small"
directories = os.listdir(dir)
if ".DS_Store" in directories:
    directories.remove(".DS_Store")
directories = [int(dir) for dir in directories]
directories.sort()
directories

In [ ]:
np.random.seed(42)
print(gc.collect())

used_angles = [-5,-4,-3,-2,-1,0,1,2,3,4,5]
if not read_small:
  used_angles = [-5, 5]

for directory in tqdm(directories):
  if directory in used_angles:
    files = os.listdir(dir + '/' + str(directory))
    files.sort()
    files = list(reversed(files))[:1100]
    for file in tqdm(files):
      try:
        distance_str = file.split('-')[0]
        distance_val = int(distance_str)
      except:
        print(f"Impossibile estrarre distanza da {file}")
        continue

      match = labels_df[(labels_df['distance'] == distance_val) & (labels_df['angle'] == directory)]
      if len(match) == 0:
        print(f"Nessuna label trovata per file {file} con distanza {distance_val} e angolo {directory}")
        continue

      train_val_test = np.random.randint(0,100)  # 80-5-15 split

      image = cv2.imread(dir + "/" + str(directory) + "/" + file)
      image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

      if train_val_test < 80:
        image = cv2.resize(image, (300,300))
        X_train.append(image.astype('float16')/255)
        Y_train.append([match.iloc[0]['distance'], match.iloc[0]['angle']])
        if len(X_train) != len(Y_train):
          print("male", file)
          print(len(X_train))
          print(len(Y_train))
      elif train_val_test < 85:
        image = cv2.resize(image, (300,300))
        X_val.append(image.astype('float16')/255)
        Y_val.append([match.iloc[0]['distance'], match.iloc[0]['angle']])
      else:
        image = cv2.resize(image, (300,300))
        X_test.append(image.astype('float16')/255)
        Y_test.append([match.iloc[0]['distance'], match.iloc[0]['angle']])
  else:
    print("Angle " + str(directory) + " skipped")


In [ ]:
gc.collect()

X_train = np.array(X_train)
Y_train = np.array(Y_train, dtype="float32")
print("Shape of Train set's images: " + str(X_train.shape))
print("Shape of Train set's labels: " + str(Y_train.shape))

X_val = np.array(X_val)
Y_val = np.array(Y_val, dtype="float32")
print("Shape of Val set's images: " + str(X_val.shape))
print("Shape of Val set's labels: " + str(Y_val.shape))

X_test = np.array(X_test)
Y_test = np.array(Y_test, dtype="float32")
print("Shape of Test set's images: " + str(X_test.shape))
print("Shape of Test set's labels: " + str(Y_test.shape))

## Labels Normalization

In [ ]:
mean = np.mean(Y_train[:,0])
std = np.std(Y_train[:,0])
Y_train[:,0] = (Y_train[:,0] - mean)/std
Y_val[:,0] = (Y_val[:,0] - mean)/std
Y_test[:,0] = (Y_test[:,0] - mean)/std

In [ ]:
np.max(Y_train[:,0]), np.min(Y_train[:,0]), np.mean(Y_train[:,0])

In [ ]:
mean, std

## Save dataset

In [ ]:
save_path = "drive/MyDrive/HPPS_Nico/HPPS/Project/ModelRegression/Dataset/TensorflowDataset"

os.makedirs(save_path, exist_ok=True)

np.save(os.path.join(save_path, "X_train.npy"), X_train)
np.save(os.path.join(save_path, "Y_train.npy"), Y_train)
np.save(os.path.join(save_path, "X_val.npy"), X_val)
np.save(os.path.join(save_path, "Y_val.npy"), Y_val)
np.save(os.path.join(save_path, "X_test.npy"), X_test)
np.save(os.path.join(save_path, "Y_test.npy"), Y_test)


## Open Dataset

In [12]:
load_path = "drive/MyDrive/HPPS_Nico/HPPS/Project/ModelRegression/Dataset/TensorflowDataset"

X_train = np.load(os.path.join(load_path, "X_train.npy"))
Y_train = np.load(os.path.join(load_path, "Y_train.npy"))
X_val = np.load(os.path.join(load_path, "X_val.npy"))
Y_val = np.load(os.path.join(load_path, "Y_val.npy"))
X_test = np.load(os.path.join(load_path, "X_test.npy"))
Y_test = np.load(os.path.join(load_path, "Y_test.npy"))


In [13]:
class OpticalDataset(torch.utils.data.Dataset):
    def __init__(self, X, Y):
        self.X = torch.tensor(X, dtype=torch.float32).unsqueeze(1)  # aggiunge il canale
        self.Y = torch.tensor(Y[:,0], dtype=torch.float32)  # solo la distanza normalizzata

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

train_dataset = OpticalDataset(X_train, Y_train)
val_dataset = OpticalDataset(X_val, Y_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [14]:
test_dataset = OpticalDataset(X_test, Y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

## Definition of the models

In [15]:
import brevitas.nn as qnn
from brevitas.quant import Int8ActPerTensorFloat

bit_width = 8

class StandardModel(nn.Module):
    def __init__(self):
        super(StandardModel, self).__init__()
        self.conv_block = nn.Sequential(
            qnn.QuantConv2d(1, 32, kernel_size=3, padding=1,
                            weight_bit_width=bit_width, bias_bit_width=bit_width,
                            input_quant=Int8ActPerTensorFloat, output_quant=Int8ActPerTensorFloat),
            qnn.QuantReLU(bit_width=bit_width),
            nn.MaxPool2d(2),

            qnn.QuantConv2d(32, 64, kernel_size=3,
                            weight_bit_width=bit_width, bias_bit_width=bit_width,
                            input_quant=Int8ActPerTensorFloat, output_quant=Int8ActPerTensorFloat),
            qnn.QuantReLU(bit_width=bit_width),
            nn.MaxPool2d(2),

            qnn.QuantConv2d(64, 128, kernel_size=3,
                            weight_bit_width=bit_width, bias_bit_width=bit_width,
                            input_quant=Int8ActPerTensorFloat, output_quant=Int8ActPerTensorFloat),
            qnn.QuantReLU(bit_width=bit_width),
            nn.MaxPool2d(2),

            qnn.QuantConv2d(128, 256, kernel_size=3, padding=1,
                            weight_bit_width=bit_width, bias_bit_width=bit_width,
                            input_quant=Int8ActPerTensorFloat, output_quant=Int8ActPerTensorFloat),
            qnn.QuantReLU(bit_width=bit_width),
            nn.MaxPool2d(2),

            qnn.QuantConv2d(256, 384, kernel_size=3,
                            weight_bit_width=bit_width, bias_bit_width=bit_width,
                            input_quant=Int8ActPerTensorFloat, output_quant=Int8ActPerTensorFloat),
            qnn.QuantReLU(bit_width=bit_width),
            nn.MaxPool2d(2),

            qnn.QuantConv2d(384, 512, kernel_size=3,
                            weight_bit_width=bit_width, bias_bit_width=bit_width,
                            input_quant=Int8ActPerTensorFloat, output_quant=Int8ActPerTensorFloat),
            qnn.QuantReLU(bit_width=bit_width)
        )
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.regressor = nn.Sequential(
            nn.Flatten(),
            qnn.QuantLinear(512, 192, weight_bit_width=bit_width, bias_bit_width=bit_width),
            qnn.QuantReLU(bit_width=bit_width),
            nn.Dropout(0.3),
            qnn.QuantLinear(192, 1, weight_bit_width=bit_width, bias_bit_width=bit_width)
        )

    def forward(self, x):
        x = self.conv_block(x)
        x = self.pool(x)
        x = self.regressor(x)
        return x


# Training

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = StandardModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.L1Loss()  # Mean Absolute Error

best_val_loss = float('inf')
patience, patience_counter = 10, 0

for epoch in range(1, 1001):
    model.train()
    train_loss = 0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device).unsqueeze(1)
        #print(f"Input shape: {inputs.shape}")
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss = loss.item()  # puoi anche accumulare per media

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device).unsqueeze(1)
            outputs = model(inputs)
            val_loss += criterion(outputs, targets).item()
    val_loss /= len(val_loader)

    print(f"Epoch {epoch:03d} - Train Loss: {train_loss:.6f} - Val Loss: {val_loss:.6f}", end='')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')
        patience_counter = 0
        print("  --> New best model saved!")
    else:
        patience_counter += 1
        print(f"  --> No improvement ({patience_counter}/{patience})")

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break


Input shape: torch.Size([32, 1, 300, 300])


/usr/local/lib/python3.11/dist-packages/torch/_tensor.py:1624: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at /pytorch/c10/core/TensorImpl.h:1935.)
  return super().rename(names)


Output streaming troncato alle ultime 5000 righe.
Input shape: torch.Size([32, 1, 300, 300])
Input shape: torch.Size([32, 1, 300, 300])
Input shape: torch.Size([32, 1, 300, 300])
Input shape: torch.Size([32, 1, 300, 300])
Input shape: torch.Size([32, 1, 300, 300])
Input shape: torch.Size([32, 1, 300, 300])
Input shape: torch.Size([32, 1, 300, 300])
Input shape: torch.Size([32, 1, 300, 300])
Input shape: torch.Size([32, 1, 300, 300])
Input shape: torch.Size([32, 1, 300, 300])
Input shape: torch.Size([32, 1, 300, 300])
Input shape: torch.Size([32, 1, 300, 300])
Input shape: torch.Size([32, 1, 300, 300])
Input shape: torch.Size([32, 1, 300, 300])
Input shape: torch.Size([32, 1, 300, 300])
Input shape: torch.Size([32, 1, 300, 300])
Input shape: torch.Size([32, 1, 300, 300])
Input shape: torch.Size([32, 1, 300, 300])
Input shape: torch.Size([32, 1, 300, 300])
Input shape: torch.Size([32, 1, 300, 300])
Input shape: torch.Size([32, 1, 300, 300])
Input shape: torch.Size([32, 1, 300, 300])
Inpu

## Save the model and collect garbage for resource issues of Colab's RAM

In [ ]:
%ls

In [ ]:
from brevitas.export import export_qonnx

class QONNXExporter:
    def __init__(self, model, model_name, input_shape, export_path):
        self.model = model
        self.model_name = model_name
        self.input_shape = input_shape
        self.export_path = export_path
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def export(self):
        self.model.to(self.device)
        self.model.eval()
        dummy_input = torch.randn(self.input_shape).to(self.device)
        export_qonnx(self.model, dummy_input, self.export_path)
        print(f"QONNX model exported to: {self.export_path}\n")


In [17]:
import os

EXPORT_DIR = "drive/MyDrive/HPPS_Nico/HPPS/Project/ModelRegression/Weights/Size300"
MODEL_NAME = "standardRegressionModel_brevitas_datasetTensorflow01_nameMappingInOrder_1100Class_80train5val15test_lr1e-4_bitWidth8_InOutQuant"

SAVE_DIR = os.path.join(EXPORT_DIR, MODEL_NAME)
os.makedirs(SAVE_DIR, exist_ok=True)


In [16]:

#model.save(os.path.join(SAVE_DIR, f"{MODEL_NAME}.keras"))
torch.save(model.state_dict(), os.path.join(SAVE_DIR, f"{MODEL_NAME}.pth"))
qonnx_export_path = os.path.join(SAVE_DIR, f"{MODEL_NAME}.qonnx")
exporter = QONNXExporter(model, MODEL_NAME, input_shape=(1, 1, 300, 300), export_path=qonnx_export_path)
exporter.export()
gc.collect()

NameError: name 'model' is not defined

## Predictions

In [20]:
from torchinfo import summary  # Assicurati che torchinfo sia installato

# Riassunto del modello
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = StandardModel().to(device)

# Riassunto del modello
summary(model, input_size=(1, 1, 300, 300))

# Dettagli layer
for name, layer in model.named_modules():
    if isinstance(layer, (nn.Conv2d, nn.Linear, nn.MaxPool2d, nn.Dropout)):
        print(f"Layer: {name}")
        if isinstance(layer, nn.Conv2d):
            print(f"  Conv2d -> Filters: {layer.out_channels}, Kernel: {layer.kernel_size}, Stride: {layer.stride}, Padding: {layer.padding}")
        if isinstance(layer, nn.Linear):
            print(f"  Linear -> Units: {layer.out_features}")
        if isinstance(layer, nn.MaxPool2d):
            print(f"  MaxPool2d -> Pool size: {layer.kernel_size}")
        if isinstance(layer, nn.Dropout):
            print(f"  Dropout -> Rate: {layer.p}")
        print()

# Predizioni su test set
model.load_state_dict(torch.load(os.path.join(SAVE_DIR, f"{MODEL_NAME}.pth")))
#model.load_state_dict(torch.load('best_model.pth'))
model.to(device)
model.eval()

with torch.no_grad():
    preds = []
    targets = []
    for inputs, target in test_loader:
        inputs = inputs.to(device)
        output = model(inputs).cpu().numpy()
        preds.extend(output)
        targets.extend(target.numpy())

preds = np.array(preds).squeeze()
targets = np.array(targets)

# Rescaling
mean = np.float32(5901007.5)
std = np.float32(633870.7)
y_test_pred_rescaled = preds * std + mean
Y_test_rescaled = targets * std + mean

# Errori
errors = np.abs(Y_test_rescaled - y_test_pred_rescaled) / 1000  # micrometri
avg_error = np.mean(errors)
max_error = np.max(errors)
min_error = np.min(errors)

# Salvataggio su file
output_text = (
    f"Average error: {avg_error} micrometers\n"
    f"Maximum error: {max_error} micrometers\n"
    f"Minimum error: {min_error} micrometers\n"
)
output_file = os.path.join(SAVE_DIR, "error_report.txt")
with open(output_file, "w") as f:
    f.write(output_text)

# Stampa
print(f"Average error is {avg_error:.6f} micrometers")
print(f"Maximum error is {max_error:.3f} micrometers")
print(f"Minimum error is {min_error:.4f} micrometers")


/usr/local/lib/python3.11/dist-packages/torch/_tensor.py:1624: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at /pytorch/c10/core/TensorImpl.h:1935.)
  return super().rename(names)


Layer: conv_block.0
  Conv2d -> Filters: 32, Kernel: (3, 3), Stride: (1, 1), Padding: (1, 1)

Layer: conv_block.2
  MaxPool2d -> Pool size: 2

Layer: conv_block.3
  Conv2d -> Filters: 64, Kernel: (3, 3), Stride: (1, 1), Padding: (0, 0)

Layer: conv_block.5
  MaxPool2d -> Pool size: 2

Layer: conv_block.6
  Conv2d -> Filters: 128, Kernel: (3, 3), Stride: (1, 1), Padding: (0, 0)

Layer: conv_block.8
  MaxPool2d -> Pool size: 2

Layer: conv_block.9
  Conv2d -> Filters: 256, Kernel: (3, 3), Stride: (1, 1), Padding: (1, 1)

Layer: conv_block.11
  MaxPool2d -> Pool size: 2

Layer: conv_block.12
  Conv2d -> Filters: 384, Kernel: (3, 3), Stride: (1, 1), Padding: (0, 0)

Layer: conv_block.14
  MaxPool2d -> Pool size: 2

Layer: conv_block.15
  Conv2d -> Filters: 512, Kernel: (3, 3), Stride: (1, 1), Padding: (0, 0)

Layer: regressor.1
  Linear -> Units: 192

Layer: regressor.3
  Dropout -> Rate: 0.3

Layer: regressor.4
  Linear -> Units: 1

Average error is 20.408615 micrometers
Maximum error is 

## Add Double Neurons

In [24]:
import copy

model_old = model  # modello originale con 1 neurone
model_new = copy.deepcopy(model_old)

# Crea nuovo layer finale a 2 neuroni
old_out_layer = model_old.regressor[-1]
new_out_layer = qnn.QuantLinear(192, 2, weight_bit_width=8, bias_bit_width=8)

with torch.no_grad():
    new_out_layer.weight[0] = old_out_layer.weight[0]
    new_out_layer.bias[0] = old_out_layer.bias[0]
    new_out_layer.weight[1].zero_()
    new_out_layer.bias[1].zero_()

# Sostituisci nell’ultimo blocco
model_new.regressor[-1] = new_out_layer

# Invia su device
model = model_new.to(device)
model.eval()


# Riassunto
from torchinfo import summary
summary(model, input_size=(1, 1, 300, 300))

for name, layer in model.named_modules():
    if isinstance(layer, (nn.Conv2d, nn.Linear, nn.MaxPool2d, nn.Dropout)):
        print(f"Layer: {name}")
        if isinstance(layer, nn.Conv2d):
            print(f"  Conv2d -> Filters: {layer.out_channels}, Kernel: {layer.kernel_size}, Stride: {layer.stride}, Padding: {layer.padding}")
        if isinstance(layer, nn.Linear):
            print(f"  Linear -> Units: {layer.out_features}")
        if isinstance(layer, nn.MaxPool2d):
            print(f"  MaxPool2d -> Pool size: {layer.kernel_size}")
        if isinstance(layer, nn.Dropout):
            print(f"  Dropout -> Rate: {layer.p}")
        print()

# Predizioni su test set usando solo il primo neurone
with torch.no_grad():
    preds = []
    targets = []
    for inputs, target in test_loader:
        inputs = inputs.to(device)
        output = model(inputs).cpu().numpy()
        preds.extend(output[:, 0])  # usa solo il primo neurone
        targets.extend(target.numpy())

preds = np.array(preds).squeeze()
targets = np.array(targets)

# Rescaling
mean = np.float32(5901007.5)
std = np.float32(633870.7)
y_test_pred_rescaled = preds * std + mean
Y_test_rescaled = targets * std + mean

# Errori
errors = np.abs(Y_test_rescaled - y_test_pred_rescaled) / 1000  # micrometri
avg_error = np.mean(errors)
max_error = np.max(errors)
min_error = np.min(errors)

# Salvataggio su file
output_text = (
    f"Average error: {avg_error} micrometers\n"
    f"Maximum error: {max_error} micrometers\n"
    f"Minimum error: {min_error} micrometers\n"
)
output_file = os.path.join(SAVE_DIR, "error_report.txt")
with open(output_file, "w") as f:
    f.write(output_text)

print(f"Average error is {avg_error:.6f} micrometers")
print(f"Maximum error is {max_error:.3f} micrometers")
print(f"Minimum error is {min_error:.4f} micrometers")


Layer: conv_block.0
  Conv2d -> Filters: 32, Kernel: (3, 3), Stride: (1, 1), Padding: (1, 1)

Layer: conv_block.2
  MaxPool2d -> Pool size: 2

Layer: conv_block.3
  Conv2d -> Filters: 64, Kernel: (3, 3), Stride: (1, 1), Padding: (0, 0)

Layer: conv_block.5
  MaxPool2d -> Pool size: 2

Layer: conv_block.6
  Conv2d -> Filters: 128, Kernel: (3, 3), Stride: (1, 1), Padding: (0, 0)

Layer: conv_block.8
  MaxPool2d -> Pool size: 2

Layer: conv_block.9
  Conv2d -> Filters: 256, Kernel: (3, 3), Stride: (1, 1), Padding: (1, 1)

Layer: conv_block.11
  MaxPool2d -> Pool size: 2

Layer: conv_block.12
  Conv2d -> Filters: 384, Kernel: (3, 3), Stride: (1, 1), Padding: (0, 0)

Layer: conv_block.14
  MaxPool2d -> Pool size: 2

Layer: conv_block.15
  Conv2d -> Filters: 512, Kernel: (3, 3), Stride: (1, 1), Padding: (0, 0)

Layer: regressor.1
  Linear -> Units: 192

Layer: regressor.3
  Dropout -> Rate: 0.3

Layer: regressor.4
  Linear -> Units: 2

Average error is 20.408615 micrometers
Maximum error is 

In [25]:
from brevitas.export import export_qonnx
import os
import gc

class QONNXExporter:
    def __init__(self, model, model_name, input_shape, export_path):
        self.model = model
        self.model_name = model_name
        self.input_shape = input_shape
        self.export_path = export_path
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def export(self):
        self.model.to(self.device)
        self.model.eval()
        dummy_input = torch.randn(self.input_shape).to(self.device)
        export_qonnx(self.model, dummy_input, self.export_path)
        print(f"QONNX model exported to: {self.export_path}\n")

# Percorsi di salvataggio
MODEL_NAME = "standardRegressionModel_brevitas_doubleNeuronsAddedPostTrain_datasetTensorflow01_nameMappingInOrder_1100Class_80train5val15test_lr1e-4_bitWidth8_InOutQuant"

SAVE_DIR = os.path.join(EXPORT_DIR, MODEL_NAME)
os.makedirs(SAVE_DIR, exist_ok=True)

# Salva il modello modificato
torch.save(model.state_dict(), os.path.join(SAVE_DIR, f"{MODEL_NAME}.pth"))

# Esporta in QONNX
qonnx_export_path = os.path.join(SAVE_DIR, f"{MODEL_NAME}.qonnx")
exporter = QONNXExporter(model, MODEL_NAME, input_shape=(1, 1, 300, 300), export_path=qonnx_export_path)
exporter.export()
gc.collect()


QONNX model exported to: drive/MyDrive/HPPS_Nico/HPPS/Project/ModelRegression/Weights/Size300/standardRegressionModel_brevitas_doubleNeuronsAddedPostTrain_datasetTensorflow01_nameMappingInOrder_1100Class_80train5val15test_lr1e-4_bitWidth8_InOutQuant/standardRegressionModel_brevitas_doubleNeuronsAddedPostTrain_datasetTensorflow01_nameMappingInOrder_1100Class_80train5val15test_lr1e-4_bitWidth8_InOutQuant.qonnx



1281